In [1]:
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from random import shuffle
import re
import pickle

In [2]:
class1_df = pd.read_pickle('./class1.pkl')
class2_df = pd.read_pickle('./class2.pkl')

In [3]:
class1_df.head()

,permid,date,headline,txt,entities,sentences,relevant_sent_list,sent_count
0,4298052532,"Friday, November 25, 2011",Report: Santander favorite to buy Caja Mediter...,Interested parties have until November 24 to s...,[Caja_Mediterráneo],[Interested parties have until November 24 to ...,"[He added, however, that ""the capital required...",4
1,4298052532,"Monday, November 28, 2011",Report: Santander favorite to buy Caja Mediter...,Interested parties have until November 24 to s...,[Caja_Mediterráneo],[Interested parties have until November 24 to ...,"[He added, however, that ""the capital required...",4
2,4298052532,"Thursday, December 01, 2011",Report: Santander favorite to buy Caja Mediter...,Interested parties have until November 24 to s...,[Caja_Mediterráneo],[Interested parties have until November 24 to ...,"[He added, however, that ""the capital required...",4
3,4298052532,"Thursday, December 08, 2011",BONDS: Europe FIG wrap: ECB liquidity move wel...,2011-12-08 News that the European Central Bank...,"[Caja_de_Ahorros, CAM]",[2011-12-08 News that the European Central Ban...,"[Of that amount, the deposit guarantee fund wi...",14
4,4298052532,"Friday, December 09, 2011",IFR European ECM Briefing - December 9 2011,Click here to see the PDF version .The EBA's f...,[Caja_de_Ahorros],[Click here to see the PDF version .The EBA's ...,[Spain's Banco de Sabadell is considering rais...,1


We would like to tag at the document- (news article) and the sentence-level.

Each new article will be tagged with "permid" and "date" (in YYYYMMDD format)
Likely, each sentence will be tagged with "permid", "date" and "sentence number"

Before we do this, we need to first check if there is more than one article discussing the same firm on the same date.

In [4]:
class1_df.groupby(['permid','date']).count()

headline  txt  entities  sentences  \
permid     date                                                                
4295859079 Wednesday, September 30, 2009         1    1         1          1   
4295865896 Friday, December 14, 2012             1    1         1          1   
           Thursday, November 15, 2012           1    1         1          1   
           Tuesday, November 13, 2012            1    1         1          1   
4295866136 Friday, November 04, 2011             1    1         1          1   
           Friday, October 21, 2011              1    1         1          1   
           Monday, October 10, 2011              3    3         3          3   
           Sunday, November 13, 2011             1    1         1          1   
           Thursday, October 13, 2011            4    4         4          4   
           Thursday, October 20, 2011            1    1         1          1   
           Tuesday, October 11, 2011             3    3         3          3   
           Wednesday, October 12, 2011           3    3         3          3   
4295866161 Monday, February 14, 2011             1    1         1          1   
           Wednesday, February 09, 2011          1    1         1          1   
4295868946 Friday, January 16, 2009              1    1         1          1   
           Friday, October 24, 2008              1    1         1          1   
           Monday, August 24, 2009               1    1         1          1   
           Monday, December 01, 2008             1    1         1          1   
           Monday, January 12, 2009              1    1         1          1   
           Monday, January 19, 2009              1    1         1          1   
           Saturday, January 17, 2009            1    1         1          1   
           Saturday, January 24, 2009            1    1         1          1   
           Saturday, November 15, 2008           1    1         1          1   
           Saturday, November 22, 2008           1    1         1          1   
           Saturday, October 25, 2008            1    1         1          1   
           Saturday, September 05, 2009          1    1         1          1   
           Thursday, September 24, 2009          1    1         1          1   
           Tuesday, December 23, 2008            1    1         1          1   
           Tuesday, January 20, 2009             1    1         1          1   
           Tuesday, September 01, 2009           1    1         1          1   
...                                            ...  ...       ...        ...   
8589934337 Monday, July 19, 2010                 4    4         4          4   
           Monday, July 26, 2010                 2    2         2          2   
           Monday, March 02, 2009                1    1         1          1   
           Monday, March 09, 2009                1    1         1          1   
           Monday, March 30, 2009                1    1         1          1   
           Saturday, July 03, 2010               1    1         1          1   
           Saturday, July 10, 2010               1    1         1          1   
           Saturday, July 17, 2010               1    1         1          1   
           Saturday, July 24, 2010               5    5         5          5   
           Saturday, July 31, 2010               1    1         1          1   
           Saturday, March 07, 2009              3    3         3          3   
           Saturday, March 28, 2009              1    1         1          1   
           Sunday, March 01, 2009                1    1         1          1   
           Thursday, July 01, 2010               1    1         1          1   
           Thursday, July 08, 2010               2    2         2          2   
           Thursday, July 15, 2010               1    1         1          1   
           Thursday, July 22, 2010               4    4         4          4   
           Thursday, July 29, 2010        

As we can see, there were 3 articles on "4295866136" on Monday, October 10, 2011. So we need to add an extra tag "x-th article of the day."

In [5]:
class1_df['date'] = pd.to_datetime(class1_df['date']) #format = "%Y%m%d")

In [6]:
class1_df = class1_df.sort_values(by=['permid','date'], ascending=[True,True])
class1_df.reset_index(drop = True, inplace= True)

In [7]:
class1_df.groupby(['permid','date']).rank()['txt']

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
5       2.0
6       3.0
7       1.0
8       2.0
9       3.0
10      2.0
11      3.0
12      1.0
13      1.0
14      4.0
15      2.0
16      3.0
17      1.0
18      1.0
19      1.0
20      1.0
21      1.0
22      1.0
23      1.0
24      1.0
25      1.0
26      1.0
27      1.0
28      1.0
29      1.0
       ... 
8183    1.0
8184    3.0
8185    3.0
8186    2.0
8187    1.0
8188    4.0
8189    2.0
8190    3.0
8191    4.0
8192    1.0
8193    3.0
8194    2.0
8195    1.0
8196    5.0
8197    3.0
8198    4.0
8199    2.0
8200    1.0
8201    1.0
8202    2.0
8203    1.0
8204    2.0
8205    2.0
8206    1.0
8207    3.0
8208    2.0
8209    1.0
8210    1.0
8211    2.0
8212    1.0
Name: txt, Length: 8213, dtype: float64

In [8]:
class1_df['doc_id'] = class1_df.groupby(['permid','date']).rank()['txt']
class1_df.head()

,permid,date,headline,txt,entities,sentences,relevant_sent_list,sent_count,doc_id
0,4295859079,2009-09-30,VEB to raise loan from German banks to finance...,MOSCOW. (Interfax) - Vnesheconombank (VEB) has...,[Volksbanken],"[MOSCOW., (Interfax) - Vnesheconombank (VEB) h...",[DZ Bank is part of the FinanzVerbund cooperat...,1,1.0
1,4295865896,2012-11-13,Report: Accountancy firms probed over Tønder B...,The Danish Business Authority launched an inve...,[Tønder_Bank_A/S],[The Danish Business Authority launched an inv...,[The Danish Business Authority launched an inv...,1,1.0
2,4295865896,2012-11-15,DK-Tønder: banking services,Document number - E361909-2012 Number of Offic...,[Tønder_Bank],[Document number - E361909-2012 Number of Offi...,[1) Type procedure Forhandling uden offentlig...,2,1.0
3,4295865896,2012-12-14,A still fragile banking sector,"A series of government rescue packages, alongs...",[Toender_Bank],"[A series of government rescue packages, along...",[It is therefore not yet clear that the risk o...,1,1.0
4,4295866136,2011-10-10,Danish Govt likely to take over Max Bank,COPENHAGEN: The Danish government's winding-up...,[Max_Bank],[COPENHAGEN: The Danish government's winding-u...,[In its annual report published on February 28...,9,1.0


In [9]:
class1_df['doc_id'] = class1_df['doc_id'].apply(lambda x: "doc_" + str(int(x)))

In [10]:
class1_df.head()

,permid,date,headline,txt,entities,sentences,relevant_sent_list,sent_count,doc_id
0,4295859079,2009-09-30,VEB to raise loan from German banks to finance...,MOSCOW. (Interfax) - Vnesheconombank (VEB) has...,[Volksbanken],"[MOSCOW., (Interfax) - Vnesheconombank (VEB) h...",[DZ Bank is part of the FinanzVerbund cooperat...,1,doc_1
1,4295865896,2012-11-13,Report: Accountancy firms probed over Tønder B...,The Danish Business Authority launched an inve...,[Tønder_Bank_A/S],[The Danish Business Authority launched an inv...,[The Danish Business Authority launched an inv...,1,doc_1
2,4295865896,2012-11-15,DK-Tønder: banking services,Document number - E361909-2012 Number of Offic...,[Tønder_Bank],[Document number - E361909-2012 Number of Offi...,[1) Type procedure Forhandling uden offentlig...,2,doc_1
3,4295865896,2012-12-14,A still fragile banking sector,"A series of government rescue packages, alongs...",[Toender_Bank],"[A series of government rescue packages, along...",[It is therefore not yet clear that the risk o...,1,doc_1
4,4295866136,2011-10-10,Danish Govt likely to take over Max Bank,COPENHAGEN: The Danish government's winding-up...,[Max_Bank],[COPENHAGEN: The Danish government's winding-u...,[In its annual report published on February 28...,9,doc_1


In [11]:
def document_tagging(class_df, which_class = "class1"):
    
    df = class_df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df.date.apply(lambda x: x.strftime('%Y%m%d'))
    df = df.sort_values(by=['permid','date'], ascending=[True,True])
    df.reset_index(drop = True, inplace = True)
    
    df['doc_ID'] = df.groupby(['permid','date']).rank()['txt']
    #df['doc_ID'] = df.doc_ID.apply(lambda x: "doc_" + str(int(x)))
    
    #entities = [e for loe in df['entities'].tolist() for e in loe]
    
    tagged_doc = []
    permid_list = []
    new_sent_list = []
    
    df['tagged_docs'] = None
    df['doc_in_words'] = None
        
    for index, row in df.iterrows():
        word_list = row['txt'].split(" ")
        word_list = [re.sub(r'[^\w\s ]','',w) for w in word_list]
        word_list = [w.lower() for w in word_list if w != '']
        tag_list = [str(row['permid']) + "_" + str(row['date']) + "_doc_" + str(int(row['doc_ID'])) + "_" +  which_class]
        permid_list.append(row['permid'])
        tagged_doc.append(TaggedDocument(words = word_list, tags = tag_list))
        
        df.set_value(index, 'tagged_docs', TaggedDocument(words = word_list, tags = tag_list))
        df.set_value(index, 'doc_in_words', word_list)
        
    return df, permid_list, tagged_doc

In [12]:
# class1_df_doc, class1_permid, class1_docs = document_tagging(class1_df)
# class2_df_doc, class2_permid, class2_docs = document_tagging(class2_df, "class2")

In [13]:
# all_articles = class1_docs + class2_docs
# all_permid = class1_permid + class2_permid

In [14]:
def sentence_tagging(class_df, which_class = 'class1', IsRelevant = False):
    df = class_df.copy()    
    
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df.date.apply(lambda x: x.strftime('%Y%m%d'))
    df = df.sort_values(by=['permid','date'], ascending=[True,True])
    df.reset_index(drop = True, inplace = True)
    
    df['doc_ID'] = df.groupby(['permid','date']).rank()['txt']
    #df['doc_ID'] = df.doc_ID.apply(lambda x: "doc_" + str(int(x)))
        
    tagged_sent = []
    permid_list = []
    
    df['sent_in_words'] = None
    df['tagged_docs'] = None
    
    for index, row in df.iterrows():
        
        if IsRelevant:
            sent_list = row['relevant_sent_list']
        else:
            sent_list = row['sentences']
        #sent_count = len(sent_list)
        i = 1
        
        sent_in_words = []
        tagged_doc = []
        
        for sent in sent_list:
            word_list = sent.split(" ")
            word_list = [re.sub(r'[^\w\s ]','',w) for w in word_list]
            word_list = [w.lower() for w in word_list if w != '']
            tag_list = [str(row['permid']) + "_" + str(row['date']) + "_doc_" + str(int(row['doc_ID'])) + "_sent_" + str(i) + "_" + which_class]
            tagged_doc.append(TaggedDocument(words = word_list, tags = tag_list))
            sent_in_words.append(word_list)
            permid_list.append(row['permid'])
            i += 1
        
        tagged_sent += tagged_doc
        
        df.set_value(index, 'sent_in_words', sent_in_words)
        df.set_value(index, 'tagged_docs', tagged_doc)
            
    return df, permid_list, tagged_sent

In [15]:
# _, class1_sent_permid, class1_sents = sentence_tagging(class1_df)
# _, class2_sent_permid, class2_sents = sentence_tagging(class2_df, "class2")

In [16]:
new_class1_df, class1_rel_sent_permid, class1_relevant_sents = sentence_tagging(class1_df, "class1", True)
new_class2_df, class2_rel_sent_permid, class2_relevant_sents = sentence_tagging(class2_df, "class2", True)

In [20]:
with open('./new_class1_df_rel.pkl', 'wb') as f:
    pickle.dump(new_class1_df, f)
    
with open('./new_class2_df_rel.pkl', 'wb') as f:
    pickle.dump(new_class2_df, f)  

In [21]:
df = pd.concat([new_class1_df, new_class2_df])
df = df.reset_index(inplace = False, drop = True)

pickle.dump(df, open('./new_rel_df.pkl', 'wb'))

In [22]:
# all_sents = class1_sents + class2_sents
all_relevant_sents = class1_relevant_sents + class2_relevant_sents

# all_sent_permid = class1_sent_permid + class2_sent_permid
all_rel_sent_permid = class1_rel_sent_permid + class2_rel_sent_permid

In [23]:
with open('./class1_relevant_sents.pkl', 'wb') as f:
    pickle.dump(class1_relevant_sents, f)
    
with open('./class2_relevant_sents.pkl', 'wb') as f:
    pickle.dump(class2_relevant_sents, f)
    
with open('./all_relevant_sents.pkl', 'wb') as f:
    pickle.dump(all_relevant_sents, f)  

with open('./class1_rel_sent_permid.pkl', 'wb') as f:
    pickle.dump(class1_rel_sent_permid, f)
    
with open('./class2_rel_sent_permid.pkl', 'wb') as f:
    pickle.dump(class2_rel_sent_permid, f)
    
with open('./all_rel_sent_permid.pkl', 'wb') as f:
    pickle.dump(all_rel_sent_permid, f)  

In [25]:
model_rel_sent = Doc2Vec(alpha=0.025, min_alpha=0.025, size = 600, window = 5)  # use fixed learning rate

model_rel_sent.build_vocab(all_relevant_sents)

model_rel_sent.train(all_relevant_sents, total_examples=model_rel_sent.corpus_count)#, epochs = 10)

model_rel_sent.save('./model_rel_sent.doc2vec')